# Python и статистика

Проект № 6 - корреляционный и регрессионный анализ.

# Введение

## Общая постановка задачи

В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные. 

## Задачи

1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (```company```). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между ```price``` и другими переменными.
4. Преобразуйте категориальные переменные с помощью ```pd.get_dummies()```. 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель?
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в $R^2$, коэффициентах и их значимости. Какую модель лучше оставить? 
7. Заполните пропуски в результатах.

# Предварительная работа с данными

## Загрузка библиотек

In [2]:
import pandas as pd

import numpy as np
import scipy as sp

import pingouin as pg

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Загрузка данных

In [3]:
cars = pd.read_csv('./data/cars.csv')

In [4]:
cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## Предобработка данных

### Проверка на дубликаты и отсутствующие значения:

In [5]:
import spelling

a0 = cars.shape[0]
a1 = cars.shape[1]
a2 = cars.dropna().shape[0]
a3 = cars.drop_duplicates().shape[0]

print(a0, spelling.rows_word(a0), "и", a1, spelling.cols_word(a1))
print(a2, spelling.rows_word(a2), "после удаления строк, содержащих NULL")
print(a3, spelling.rows_word(a3), "после удаления дубликатов")

205 строк и 26 колонок
205 строк после удаления строк, содержащих NULL
205 строк после удаления дубликатов


### Проверка типов данных

In [6]:
cars.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [7]:
cars.columns = cars.columns.str.lower()

In [8]:
cars.columns

Index(['car_id', 'symboling', 'carname', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

# Решение поставленных задач

### Добавление нового признака

Создадим новый признак – марку автомобиля (company).

Посмотрим, машины каких производителей встречаются в датасете и, при необходимости, исправим названия и проверим изменения.

In [9]:
cars['company'] = cars['carname'].str.split().apply(lambda lst: lst[0])

In [10]:
cars['company'].nunique()

28

In [11]:
cars = cars.drop(['car_id', 'carname'], axis = 1)

In [12]:
cars['company'].value_counts()

toyota         31
nissan         17
mazda          15
mitsubishi     13
honda          13
subaru         12
peugeot        11
volvo          11
dodge           9
volkswagen      9
bmw             8
buick           8
audi            7
plymouth        7
saab            6
porsche         4
isuzu           4
chevrolet       3
alfa-romero     3
jaguar          3
maxda           2
renault         2
vw              2
porcshce        1
toyouta         1
vokswagen       1
mercury         1
Nissan          1
Name: company, dtype: int64

In [13]:
cars['company'] = \
    cars['company'] \
    .str.replace('maxda', 'mazda') \
    .str.replace('Nissan', 'nissan') \
    .str.replace('porcshce', 'porsche') \
    .str.replace('toyouta', 'toyota') \
    .str.replace('vokswagen', 'volkswagen') \
    .str.replace('vw', 'volkswagen')

In [14]:
cars['company'].value_counts().shape[0]

22

### Корреляция между ценой и другими признаками

Посчитаем корреляцию между `price` и другими переменными. Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

In [15]:
cars = cars[['price',
             'company',
             'fueltype',
             'aspiration',
             'carbody',
             'drivewheel',
             'wheelbase',
             'carlength',
             'carwidth',
             'curbweight',
             'enginetype',
             'cylindernumber',
             'enginesize',
             'boreratio',
             'horsepower']]

In [16]:
# Постройка матрицы корреляций. По умолчанию - вычисляется коэффициент Пирсона.
cars.corr()

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.000000,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139
wheelbase,0.577816,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294
carlength,0.682920,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623
carwidth,0.759325,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732
curbweight,0.835305,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739
enginesize,0.874145,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769
boreratio,0.553173,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677
horsepower,0.808139,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000


Корреляция между `price` и `horsepower`:

In [17]:
round(cars.corr().loc['price', 'horsepower'], 2)

0.81

### Регрессионный анализ

Линейная регрессия в **python** не справляется с категориальными переменными (типом `object` в **pandas**), поэтому применим `get_dummies()`.

In [18]:
cars.dtypes[cars.dtypes == 'object'].index.to_list()

['company',
 'fueltype',
 'aspiration',
 'carbody',
 'drivewheel',
 'enginetype',
 'cylindernumber']

In [19]:
cars.dtypes[cars.dtypes != 'object'].index.to_list()

['price',
 'wheelbase',
 'carlength',
 'carwidth',
 'curbweight',
 'enginesize',
 'boreratio',
 'horsepower']

In [20]:
cars_dummies = \
    pd.concat([cars[cars.dtypes[cars.dtypes != 'object'].index.to_list()],
               pd.get_dummies(data = cars[cars.dtypes[cars.dtypes == 'object'].index.to_list()],
                              drop_first = True)
              ],
              axis = 1)

cars_dummies

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,13495.0,88.6,168.8,64.1,2548,130,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
1,16500.0,88.6,168.8,64.1,2548,130,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16500.0,94.5,171.2,65.5,2823,152,2.68,154,0,0,...,0,0,1,0,0,0,1,0,0,0
3,13950.0,99.8,176.6,66.2,2337,109,3.19,102,1,0,...,1,0,0,0,0,1,0,0,0,0
4,17450.0,99.4,176.6,66.4,2824,136,3.19,115,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,109.1,188.8,68.9,2952,141,3.78,114,0,0,...,1,0,0,0,0,1,0,0,0,0
201,19045.0,109.1,188.8,68.8,3049,141,3.78,160,0,0,...,1,0,0,0,0,1,0,0,0,0
202,21485.0,109.1,188.8,68.9,3012,173,3.58,134,0,0,...,0,0,1,0,0,0,1,0,0,0
203,22470.0,109.1,188.8,68.9,3217,145,3.01,106,0,0,...,1,0,0,0,0,0,1,0,0,0


#### Анализ с одним предиктором (`horsepower`)

In [21]:
# Y = одномерный массив с ЗП, X - массив с НП

# добавить константу, чтобы был свободный член
X_horsepower = sm.add_constant(cars_dummies['horsepower'].to_numpy())

# говорим модели, что у нас ЗП, а что НП
model_horsepower = sm.OLS(cars_dummies['price'].to_numpy(), X_horsepower)

results_horsepower = model_horsepower.fit()  # строим регрессионную прямую
print(results_horsepower.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 23 May 2022   Prob (F-statistic):           1.48e-48
Time:                        11:40:58   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.0

#### Анализ без предикторов, указывающих на марку

In [22]:
# Y = одномерный массив с ЗП, X - массив с НП

# добавить константу, чтобы был свободный член
X_nocompanies = \
    sm.add_constant(cars_dummies \
                        .drop(['price'] + \
                              cars_dummies.columns[cars_dummies.columns.str.startswith('company')].to_list(),
                              axis = 1) \
                        .to_numpy())

# говорим модели, что у нас ЗП, а что НП
model_nocompanies = sm.OLS(cars_dummies['price'].to_numpy(), X_nocompanies)

results_nocompanies = model_nocompanies.fit()  # строим регрессионную прямую
print(results_nocompanies.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Mon, 23 May 2022   Prob (F-statistic):           9.86e-81
Time:                        11:40:58   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -1.7e+04    1.3e+04     -1.309      0.1

#### Анализ со всеми предикторами

In [23]:
# Y = одномерный массив с ЗП, X - массив с НП

# добавить константу, чтобы был свободный член
X = sm.add_constant(cars_dummies.drop('price', axis = 1).to_numpy())

# говорим модели, что у нас ЗП, а что НП
model = sm.OLS(cars_dummies['price'].to_numpy(), X)

results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Mon, 23 May 2022   Prob (F-statistic):           4.86e-89
Time:                        11:40:58   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.472e+04   1.17e+04     -2.959      0.0

Из двух моделей (со всеми предикторами (1) и почти всеми, кроме марки машины (2)) оставляем ту, где меньше предикторов, так как **R2** изменился не очень сильно, а часть марок вообще не значима. Выбранная модель объясняет примерно 90 % дисперсии. Среди предикторов 10 из 27 оказались не значимыми (p > 0.05).

Пример интерпретации: при единичном изменении показателя `horsepower` цена увеличивается на 86.82.